In [1]:
!pip install skforecast tqdm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 560.6/560.6 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 380.1/380.1 kB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.0/233.0 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 9.6 MB/s eta 0:00:00


In [2]:
import pandas as pd
import lightgbm
import sklearn
from lightgbm import LGBMRegressor
from sklearn.linear_model import LinearRegression
from sklearn.impute import KNNImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import FunctionTransformer
from sklearn.pipeline import make_pipeline
from sklearn.compose import make_column_transformer
from sklearn.compose import make_column_selector
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import PolynomialFeatures
import skforecast
from sklearn.feature_selection import RFECV
from skforecast.ForecasterAutoreg import ForecasterAutoreg
from skforecast.model_selection import bayesian_search_forecaster
from skforecast.model_selection import backtesting_forecaster
from skforecast.model_selection import select_features
import matplotlib.pyplot as plt
import numpy as np
import os
from os import path
import shutil
import re
import traceback
%matplotlib inline

In [3]:
pipeline_categorical = make_pipeline(
                           OrdinalEncoder(
                               dtype=int,
                               handle_unknown="use_encoded_value",
                               unknown_value=-1,
                               encoded_missing_value=-1
                           ),
                           FunctionTransformer(
                               func=lambda x: x.astype('category'),
                               feature_names_out= 'one-to-one'
                           )
                       )

transformer_exog = make_column_transformer(
                       (
                           pipeline_categorical,
                           make_column_selector(dtype_exclude=np.number)
                       ),
                       remainder="passthrough",
                       verbose_feature_names_out=False,
                   ).set_output(transform="pandas")

# Lags grid
lags_grid = tuple([12, 24, [1, 2, 3, 4, 7, 9, 24]])

# Regressor hyperparameters search space
def search_space(trial):
    search_space  = {
        'n_estimators'    : trial.suggest_int('n_estimators', 400, 1200, step=100),
        'max_depth'       : trial.suggest_int('max_depth', 3, 10, step=1),
        'min_data_in_leaf': trial.suggest_int('min_data_in_leaf', 25, 500),
        'learning_rate'   : trial.suggest_float('learning_rate', 0.01, 0.5),
        'feature_fraction': trial.suggest_float('feature_fraction', 0.5, 1, step=0.1),
        'max_bin'         : trial.suggest_int('max_bin', 50, 250, step=25),
        'reg_alpha'       : trial.suggest_float('reg_alpha', 0, 1, step=0.1),
        'reg_lambda'      : trial.suggest_float('reg_lambda', 0, 1, step=0.1),
        'lags'            : trial.suggest_categorical('lags', lags_grid)
    }
    return search_space

In [4]:
def search_hyperparameters(data, end_train, end_valid, exog_features,transformer_exog):

  # instantiate a forcaster transformer with categorical features
    forecaster = ForecasterAutoreg(
        regressor = LGBMRegressor(random_state=15926, verbose=-1),
        lags = 24,
        transformer_exog = transformer_exog,
        fit_kwargs = {"categorical_feature": "auto"}
    )

    # search for best parameters
    results_search, frozen_trial = bayesian_search_forecaster(
    forecaster         = forecaster,
    y                  = data.loc[:end_valid, 'gw-level'],
    exog               = data.loc[:end_valid, exog_features],
    search_space       = search_space,
    steps              = 30,
    refit              = False,
    metric             = 'mean_absolute_percentage_error',
    initial_train_size = len(data.loc[:end_train]),
    fixed_train_size   = False,
    n_trials           = 20,
    random_state       = 123,
    return_best        = True,
    n_jobs             = 'auto',
    verbose            = False,
    show_progress      = True
    )

    best_params = results_search['params'].iat[0]

    return best_params, forecaster

def train_and_predict(data, best_params, actual_data, end_valid, end_train, valid_num, train_num, df_idx, exog_features, transformer_exog, hrbnz01):

  # train for evaluation of the model
  forecaster = ForecasterAutoreg(
  regressor          = LGBMRegressor(**best_params, random_state=15926, verbose=-1),
  lags               = 24,
  transformer_exog   = transformer_exog,
  fit_kwargs         = {"categorical_feature": "auto"}
  )
  # train the model the time series train and validation dataset
  forecaster.fit(
    y    = data.loc[:end_train, 'gw-level'],
    exog = data.loc[:end_train, exog_features]
  )

  # make predictions and evalute the model
  predictions = forecaster.predict(
      exog     = data.loc[df_idx[train_num+1]:, exog_features],
      steps    = 26
  )
  df_preds = pd.DataFrame(predictions)
  preds = df_preds["pred"].values
  # evaluate on symmetric mean absolute percentage error
  smape_value = smape(actual_data, preds)

  # train for future predictions
  forecaster = ForecasterAutoreg(
  regressor          = LGBMRegressor(**best_params, random_state=15926, verbose=-1),
  lags               = 24,
  transformer_exog   = transformer_exog, #one_hot_encoder,
  fit_kwargs         = {"categorical_feature": "auto"}
  )
  # train the model the time series train and validation dataset
  forecaster.fit(
    y    = data.loc[:end_valid, 'gw-level'],
    exog = data.loc[:end_valid, exog_features]
  )

  # make predictions into the future
  predictions = forecaster.predict(
    exog     = data.loc[df_idx[valid_num+1]:, exog_features],
    steps    = 26
  )
  df_preds = pd.DataFrame(predictions)

  # free resources since it's going to run on iterations
  del forecaster

  return df_preds, smape_value


In [5]:
# Define the function to return the SMAPE value
def smape(A, F):
    tmp = 2 * np.abs(F - A) / (np.abs(A) + np.abs(F))
    len_ = np.count_nonzero(~np.isnan(tmp))
    if len_ == 0 and np.nansum(tmp) == 0: # Deals with a special case
        return 100
    return round(100 / len_ * np.nansum(tmp), 3)

In [6]:
from tqdm import tqdm
def populate_test_data(data_dir):

  preds_dict = {}
  smape_dict = {}


  # collect all files in the directory
  filenames = os.listdir(data_dir)
  filenames = filenames[:10]

  try:

    for filename in tqdm(filenames):

        hrbnz01 = filename.split(".")[0].split("-")[-1]
        filepath = path.join(data_dir, filename)
        df_exog = pd.read_csv(filepath)
        df_exog["season"] = df_exog["season"].astype("category")
        df_exog["weather"] = df_exog["weather"].astype("category")
        df_exog["date"] = pd.to_datetime(df_exog["date"])
        df_exog.set_index("date", inplace=True)
        df_exog.index = pd.date_range(start=df_exog.index.min(), end=df_exog.index.max(), freq='MS')

        # get the estimate end train and end validation dates
        data = df_exog.copy()
        exog_data = data.drop("gw-level", axis=1)
        exog_features = exog_data.columns
        df_idx = data.index
        train_num = int(len(data) * 0.8)
        valid_num = len(data.loc[:"2021-11-01"])
        end_train = df_idx[train_num]
        end_valid = df_idx[valid_num]
        end_evaluation = df_idx[train_num+26]
        evaluate_data = data.loc[df_idx[train_num+1]: end_evaluation, "gw-level"].values


        # tune for best hyperparamters and evaluate on MAPE metric
        best_params, metric = search_hyperparameters(data, end_train, end_valid, exog_features, transformer_exog)

        # train and make predict into 26 months in the future of the test template
        df_predictions, smape = train_and_predict(data,best_params, evaluate_data, end_valid, end_train, valid_num, train_num, df_idx, exog_features, transformer_exog, hrbnz01)
        df_predictions["pred"] =  df_predictions["pred"].round(2)
        preds_dict[hrbnz01] = df_predictions['pred'].values
        smape_dict[hrbnz01] = smape


  except Exception as ex:
    print("[Error]")
    print(traceback.format_exc())

  df_smape = pd.DataFrame(smape_dict, index=[0])
  df_final_preds = pd.DataFrame(preds_dict, index=range(26))
  print("> Done")

  return df_final_preds, df_smape

In [7]:
processed_data_dir = "/content/drive/MyDrive/processed_data_part1"
test_template = "/content/drive/MyDrive/gw_test_empty.csv"

df_submission, df_smape = populate_test_data(processed_data_dir)
df_submission.to_csv("df_submission.csv", index=False)
df_smape.to_csv("smape_score.csv", index=False)

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/20 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/optuna/distributions.py:524: UserWarning: Choices for a categorical distribution should be a tuple of None, bool, int, float and str for persistent storage but contains [1, 2, 3, 4, 7, 9, 24] which is of type list.
  warnings.warn(message)
/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_discretization.py:245: UserWarning: Feature 0 is constant and will be replaced with 0.
  warnings.warn(


`Forecaster` refitted using the best-found lags and parameters, and the whole data set: 
  Lags: [ 1  2  3  4  5  6  7  8  9 10 11 12] 
  Parameters: {'n_estimators': 600, 'max_depth': 9, 'min_data_in_leaf': 408, 'learning_rate': 0.05151409102341216, 'feature_fraction': 1.0, 'max_bin': 250, 'reg_alpha': 0.4, 'reg_lambda': 0.1}
  Backtesting metric: 0.0028209927694297756



 10%|█         | 1/10 [00:08<01:14,  8.33s/it]

...

/usr/local/lib/python3.10/dist-packages/optuna/distributions.py:524: UserWarning: Choices for a categorical distribution should be a tuple of None, bool, int, float and str for persistent storage but contains [1, 2, 3, 4, 7, 9, 24] which is of type list.
  warnings.warn(message)


`Forecaster` refitted using the best-found lags and parameters, and the whole data set: 
  Lags: [ 1  2  3  4  7  9 24] 
  Parameters: {'n_estimators': 900, 'max_depth': 3, 'min_data_in_leaf': 25, 'learning_rate': 0.09266753030469671, 'feature_fraction': 0.6, 'max_bin': 225, 'reg_alpha': 0.30000000000000004, 'reg_lambda': 0.0}
  Backtesting metric: 0.0009230126427751498



100%|██████████| 10/10 [01:00<00:00,  6.06s/it]

> Done



<frozen importlib._bootstrap>:914: ImportWarning: _PyDrive2ImportHook.find_spec() not found; falling back to find_module()
<frozen importlib._bootstrap>:914: ImportWarning: _PyDriveImportHook.find_spec() not found; falling back to find_module()
<frozen importlib._bootstrap>:914: ImportWarning: _GenerativeAIImportHook.find_spec() not found; falling back to find_module()
<frozen importlib._bootstrap>:914: ImportWarning: _OpenCVImportHook.find_spec() not found; falling back to find_module()
<frozen importlib._bootstrap>:914: ImportWarning: APICoreClientInfoImportHook.find_spec() not found; falling back to find_module()
<frozen importlib._bootstrap>:914: ImportWarning: _BokehImportHook.find_spec() not found; falling back to find_module()
<frozen importlib._bootstrap>:914: ImportWarning: _AltairImportHook.find_spec() not found; falling back to find_module()


In [11]:
smp =pd.read_csv("/content/smape_score.csv")
print(smp.mean(axis=1))

0    0.1273
dtype: float64


In [12]:
df = pd.read_csv("/content/df_submission.csv")
df.index = pd.date_range(start=pd.Timestamp("2021-12-01") + pd.DateOffset(months=1), periods=26, freq='MS')
df

,300111,345710,329078,301838,345116,345512,326934,321646,335208,309625
2022-01-01,153.89,399.97,238.26,200.74,224.31,230.86,242.54,510.75,328.43,492.98
2022-02-01,153.89,400.00,238.26,200.71,224.38,230.76,242.46,510.81,328.39,493.27
2022-03-01,153.99,399.97,238.26,200.72,224.33,230.61,242.53,510.88,328.38,493.44
2022-04-01,154.02,399.99,238.26,200.68,224.22,230.51,242.84,511.05,328.16,493.66
2022-05-01,154.00,400.00,238.26,200.66,224.40,230.80,243.14,511.18,328.39,493.41
2022-06-01,153.95,400.08,238.26,200.62,224.50,230.90,243.35,511.21,328.42,492.76
2022-07-01,153.86,400.16,238.26,200.64,224.60,230.99,243.30,511.08,328.54,492.57
2022-08-01,153.86,400.16,238.26,200.63,224.56,230.98,243.28,511.01,328.89,492.93
2022-09-01,153.85,400.16,238.26,200.65,224.50,230.91,243.18,510.93,329.08,493.20
2022-10-01,153.76,400.16,238.26,200.70,224.51,231.07,242.53,510.87,329.16,493.27


## This is code for a single location prediction

In [19]:
df_exog = pd.read_csv("/content/drive/MyDrive/processed_data/Kärnten-309641.csv")

# set categorical columns astype category for the transformer model to auto detect them
df_exog["season"] = df_exog["season"].astype("category")
df_exog["weather"] = df_exog["weather"].astype("category")
df_exog["date"] = pd.to_datetime(df_exog["date"])
df_exog.set_index("date", inplace=True)
# set the datatime range index to monthly freq
df_exog.index = pd.date_range(start=df_exog.index.min(), end=df_exog.index.max(), freq='MS')
df_exog

,gw-level,month,year,quarter,season,weather,month_sin,month_cos,quarter_sin,quarter_cos,...,poly_quarter_month_sin,poly_quarter_month_cos,poly_quarter_quarter_sin,poly_quarter_quarter_cos,poly_month_sin_month_cos,poly_month_sin_quarter_sin,poly_month_sin_quarter_cos,poly_month_cos_quarter_sin,poly_month_cos_quarter_cos,poly_quarter_sin_quarter_cos
1980-01-01,493.71,1,1980,1,winter,cold,2.588190e-01,0.965926,1.000000e+00,6.123234e-17,...,2.588190e-01,0.965926,1.000000e+00,6.123234e-17,2.500000e-01,2.588190e-01,1.584810e-17,9.659258e-01,5.914590e-17,6.123234e-17
1980-02-01,493.64,2,1980,1,winter,cold,5.000000e-01,0.866025,1.000000e+00,6.123234e-17,...,5.000000e-01,0.866025,1.000000e+00,6.123234e-17,4.330127e-01,5.000000e-01,3.061617e-17,8.660254e-01,5.302876e-17,6.123234e-17
1980-03-01,493.54,3,1980,1,spring,normal,7.071068e-01,0.707107,1.000000e+00,6.123234e-17,...,7.071068e-01,0.707107,1.000000e+00,6.123234e-17,5.000000e-01,7.071068e-01,4.329780e-17,7.071068e-01,4.329780e-17,6.123234e-17
1980-04-01,493.64,4,1980,2,spring,normal,8.660254e-01,0.500000,1.224647e-16,-1.000000e+00,...,1.732051e+00,1.000000,2.449294e-16,-2.000000e+00,4.330127e-01,1.060575e-16,-8.660254e-01,6.123234e-17,-5.000000e-01,-1.224647e-16
1980-05-01,493.92,5,1980,2,spring,normal,9.659258e-01,0.258819,1.224647e-16,-1.000000e+00,...,1.931852e+00,0.517638,2.449294e-16,-2.000000e+00,2.500000e-01,1.182918e-16,-9.659258e-01,3.169619e-17,-2.588190e-01,-1.224647e-16
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-10-01,NaN,10,2023,4,Fall,normal,5.000000e-01,-0.866025,-2.449294e-16,1.000000e+00,...,2.000000e+00,-3.464102,-9.797174e-16,4.000000e+00,-4.330127e-01,-1.224647e-16,5.000000e-01,2.121150e-16,-8.660254e-01,-2.449294e-16
2023-11-01,NaN,11,2023,4,Fall,normal,2.588190e-01,-0.965926,-2.449294e-16,1.000000e+00,...,1.035276e+00,-3.863703,-9.797174e-16,4.000000e+00,-2.500000e-01,-6.339238e-17,2.588190e-01,2.365836e-16,-9.659258e-01,-2.449294e-16
2023-12-01,NaN,12,2023,4,winter,cold,1.224647e-16,-1.000000,-2.449294e-16,1.000000e+00,...,4.898587e-16,-4.000000,-9.797174e-16,4.000000e+00,-1.224647e-16,-2.999520e-32,1.224647e-16,2.449294e-16,-1.000000e+00,-2.449294e-16
2024-01-01,NaN,1,2024,1,winter,cold,2.588190e-01,0.965926,1.000000e+00,6.123234e-17,...,2.588190e-01,0.965926,1.000000e+00,6.123234e-17,2.500000e-01,2.588190e-01,1.584810e-17,9.659258e-01,5.914590e-17,6.123234e-17


In [20]:
pipeline_categorical = make_pipeline(
                           OrdinalEncoder(
                               dtype=int,
                               handle_unknown="use_encoded_value",
                               unknown_value=-1,
                               encoded_missing_value=-1
                           ),
                           FunctionTransformer(
                               func=lambda x: x.astype('category'),
                               feature_names_out= 'one-to-one'
                           )
                       )

transformer_exog = make_column_transformer(
                       (
                           pipeline_categorical,
                           make_column_selector(dtype_exclude=np.number)
                       ),
                       remainder="passthrough",
                       verbose_feature_names_out=False,
                   ).set_output(transform="pandas")

# Lags grid
lags_grid = tuple([12, 24, [1, 2, 3, 4, 7, 9, 24]])

# Regressor hyperparameters search space
def search_space(trial):
    search_space  = {
        'n_estimators'    : trial.suggest_int('n_estimators', 400, 1200, step=100),
        'max_depth'       : trial.suggest_int('max_depth', 3, 10, step=1),
        'min_data_in_leaf': trial.suggest_int('min_data_in_leaf', 25, 500),
        'learning_rate'   : trial.suggest_float('learning_rate', 0.01, 0.5),
        'feature_fraction': trial.suggest_float('feature_fraction', 0.5, 1, step=0.1),
        'max_bin'         : trial.suggest_int('max_bin', 50, 250, step=25),
        'reg_alpha'       : trial.suggest_float('reg_alpha', 0, 1, step=0.1),
        'reg_lambda'      : trial.suggest_float('reg_lambda', 0, 1, step=0.1),
        'lags'            : trial.suggest_categorical('lags', lags_grid)
    }
    return search_space

In [28]:
def search_hyperparameters(data, one_hot_encoder):

    # get the estimate end train and end validation dates
    exog_data = data.drop("gw-level", axis=1)
    exog_features = exog_data.columns
    df_idx = data.index
    train_num = int(len(data) * 0.8)
    end_train = "2012-12-01"
    end_valid = "2021-12-01"

    # instantiate a forcaster transformer with categorical features
    forecaster = ForecasterAutoreg(
        regressor = LGBMRegressor(random_state=15926, verbose=-1),
        lags = 24,
        transformer_exog = one_hot_encoder,
        fit_kwargs = {"categorical_feature": "auto"}
    )

    # search for best parameters
    results_search, frozen_trial = bayesian_search_forecaster(
    forecaster         = forecaster,
    y                  = data.loc[:end_valid, 'gw-level'],
    exog               = data.loc[:end_valid, exog_features],
    search_space       = search_space,
    steps              = 30,
    refit              = False,
    metric             = 'mean_absolute_percentage_error',
    initial_train_size = len(data.loc[:end_train]),
    fixed_train_size   = False,
    n_trials           = 20,
    random_state       = 123,
    return_best        = True,
    n_jobs             = 'auto',
    verbose            = False,
    show_progress      = True
    )
    best_params = results_search['params'].iat[0]

    return best_params, forecaster

best_params, forecaster =  search_hyperparameters(df_exog, transformer_exog)

  0%|          | 0/20 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/optuna/distributions.py:524: UserWarning: Choices for a categorical distribution should be a tuple of None, bool, int, float and str for persistent storage but contains [1, 2, 3, 4, 7, 9, 24] which is of type list.
  warnings.warn(message)


`Forecaster` refitted using the best-found lags and parameters, and the whole data set: 
  Lags: [ 1  2  3  4  5  6  7  8  9 10 11 12] 
  Parameters: {'n_estimators': 400, 'max_depth': 3, 'min_data_in_leaf': 75, 'learning_rate': 0.019330721332114376, 'feature_fraction': 0.8, 'max_bin': 50, 'reg_alpha': 0.8, 'reg_lambda': 0.30000000000000004}
  Backtesting metric: 0.0007770916999934754



In [29]:
data = df_exog.copy()
exog_data = data.drop("gw-level", axis=1)
exog_features = exog_data.columns
df_idx = data.index
train_num = int(len(data) * 0.8)
valid_num = len(data.loc[:"2021-11-01"])
end_train = df_idx[train_num]
end_valid = df_idx[valid_num]
end_evaluation = df_idx[train_num+50]
evaluate_data = data.loc[df_idx[train_num+1]: end_evaluation, "gw-level"].values

In [30]:
# train model with best params
forecaster = ForecasterAutoreg(
    regressor          = LGBMRegressor(**best_params, random_state=15926, verbose=-1),
    lags               = 24,
    transformer_exog   = transformer_exog,
    fit_kwargs         = {"categorical_feature": "auto"}
)
forecaster.fit(
    y    = data.loc[:end_train, 'gw-level'],
    exog = data.loc[:end_train, exog_features]
)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [31]:
# make predictions and evalute the model
predictions = forecaster.predict(
    exog     = data.loc[df_idx[train_num+1]:, exog_features],
    steps    = 50
)
pred_df = pd.DataFrame(predictions)

In [32]:
preds = pred_df["pred"].values
smape_value = smape(evaluate_data, preds)
pred_df.loc["smape", "pred"] = smape_value
pred_df

,pred
2015-06-01 00:00:00,493.294491
2015-07-01 00:00:00,493.324205
2015-08-01 00:00:00,493.367466
2015-09-01 00:00:00,493.520476
2015-10-01 00:00:00,493.931530
2015-11-01 00:00:00,494.369372
2015-12-01 00:00:00,494.612824
2016-01-01 00:00:00,494.223980
2016-02-01 00:00:00,494.133360
2016-03-01 00:00:00,494.198262


In [ ]:
# make predictions into the future
predictions = forecaster.predict(
    exog     = data.loc[df_idx[valid_num+1]:, exog_features],
    steps    = 24
)
pd.DataFrame(predictions)

,pred
2022-01-01,260.203161
2022-02-01,260.347878
2022-03-01,260.364267
2022-04-01,260.348622
2022-05-01,260.375187
2022-06-01,260.378496
2022-07-01,260.315726
2022-08-01,260.353924
2022-09-01,260.271444
2022-10-01,260.182519
